In [1]:
import h5py
import tensorflow as tf
import numpy as np
from numpy import f2py

## Wrapping Fortran functions to Python

[Three ways to wrap](https://numpy.org/doc/stable/f2py/f2py.getting-started.html)

- Quick way (this requires Python 3.7 and NumPy 1.18): `python3 -m numpy.f2py -c eos-t.f90 -m eos_t`


In [3]:
from platform import python_version
print(python_version())

3.7.10


In [4]:
import eos_t

In [5]:
eos_t.atomic_rates.tabulate_rates()
print(eos_t.atomic_rates.alphahp)

[1.26271073e-10 1.25558019e-10 1.24848791e-10 ... 6.57445059e-19
 6.47458444e-19 6.37622769e-19]


# TensorFlow

In [2]:
# testing element-wise division
x = [1,1]
y = tf.constant([2.,2.])
tf.math.divide(x,y)

<tf.Tensor: shape=(2,), dtype=float32, numpy=array([0.5, 0.5], dtype=float32)>

## Assigning an element to an existing tensor

In [7]:
x = tf.zeros([3,3,3])
#tf.tensor_scatter_nd_add?

In [6]:
skewer = tf.zeros([3])
for i in range(3):
    for j in range(3):
        x = tf.tensor_scatter_nd_add(x, [[i,j]], [skewer])
        skewer = skewer + 1
x

<tf.Tensor: shape=(3, 3, 3), dtype=float32, numpy=
array([[[0., 0., 0.],
        [1., 1., 1.],
        [2., 2., 2.]],

       [[3., 3., 3.],
        [4., 4., 4.],
        [5., 5., 5.]],

       [[6., 6., 6.],
        [7., 7., 7.],
        [8., 8., 8.]]], dtype=float32)>

In [12]:
x = tf.zeros([3])
tf.tensor_scatter_nd_add(x, [[1]], [3])

<tf.Tensor: shape=(3,), dtype=float32, numpy=array([0., 3., 0.], dtype=float32)>

## Testing speed of tensor access

It takes too long to access a tensor's entries one-by-one; I need to avoid this when I'm passing fields into the EOS or optical depth routines

In [2]:
def time_stats(duration):
    rate = duration / n**2
    print("Duration:", np.round(duration, 4))
    print("Hours needed to access 1024^3 entries:", 
      np.round(rate * 1024**3 / 3600, 4))

In [4]:
import time

n = 5
field = tf.zeros((n,n))
total = 0

start = time.time()

for i in range(n):
    for j in range(n):
        total += field[i,j]

duration = time.time() - start
time_stats(duration)

Duration: 0.0085
Hours needed to access 1024^3 entries: 101.1769


### Nested loops

In [24]:
count = 15
n = 10

start = time.time()

# outer loop
i = tf.constant(0)
condition1 = lambda i, count: tf.less(i, n)

def body1(i, r): # index i and result r
    # inner loop
    j = tf.constant(0)
    condition2 = lambda i,j,r: tf.less(j, n)
    
    def body2(i, j, r):
        r += field[i,j]
        return i, tf.add(j, 1), r
    
    i, j, r = tf.while_loop(condition2, body2, loop_vars=[i,j, r])
    
    # increment r
    return tf.add(i, 1), r

# do the loop:
r = tf.while_loop(condition1, body1, [i, count])
print(r)

duration = time.time() - start
time_stats(duration)

(<tf.Tensor: shape=(), dtype=int32, numpy=10>, <tf.Tensor: shape=(), dtype=float32, numpy=15.0>)
Duration: 0.0686
Hours needed to access 1024^3 entries: 204.7353


## Testing tf gradients

In [13]:
tf.math.pow(z, 2)

<tf.Tensor: shape=(), dtype=float32, numpy=9.0>

In [11]:
z = tf.constant(3.)
with tf.GradientTape() as tape:
    tape.watch(z) # without this line, grad is None
    a = tf.divide(1, z+1)
    print('a:', a)
    
grad = tape.gradient(a, z)
print('grad:', grad)

a: tf.Tensor(0.25, shape=(), dtype=float32)
grad: tf.Tensor(-0.0625, shape=(), dtype=float32)


# Load in the data

In [1]:
filename = "../../../../../cscratch1/sd/jupiter/sim2_z3_FGPA_cgs.h5"

#snap = h5py.File(filename,'r')

In [5]:
name = 'aux_fields'

snap.keys()

if name in snap:
    print('ok')

ok


## Testing classes

In [2]:
import snapshot
import time

snap = snapshot.Snapshot(filename)

In [3]:
# test snapshot methods
start = time.time()

temp = snap.read_field('/native_fields/temperature')
rhob = snap.read_field('/native_fields/baryon_density')
print(temp.shape)
print(temp.size)
print('Time:', time.time() - start)

[1024 1024 1024]
[21.09375 21.09375 21.09375]
Time: 11.72275996208191


In [3]:
# test read_field2 (reading in n^3 grids)
start = time.time()
shape = [5,5,5]

temp2 = snap.read_field2('/native_fields/temperature', shape)
print(temp2.shape)
print(temp2.size)
print('Time:', time.time() - start)

[5, 5, 5]
tf.Tensor([0.10299683 0.10299683 0.10299683], shape=(3,), dtype=float64)
Time: 17.140169620513916


### universe

In [3]:
u = snap.universe
chi = 10
z = 1
u.chi_to_proper_cgs(chi, z)

<tf.Tensor: shape=(), dtype=float64, numpy=2.2856888888888885e+25>

In [4]:
u.h

<tf.Variable 'Variable:0' shape=() dtype=float64, numpy=0.675>

### eos

In [5]:
import eos

eos_obj = eos.EOS_at_z(snap.z)
# returns nhi
eos_obj.nyx_eos(rhob.field[0,0,0] * 1e-29, temp.field[0,0,0])

9.275646225472467e-13

## Metadata

In [5]:
shape = snap['domain'].attrs['shape']
size = snap['domain'].attrs['size']

z = snap['universe'].attrs['redshift']
omega_b = snap['universe'].attrs['omega_b']
omega_m = snap['universe'].attrs['omega_m']
omega_l = snap['universe'].attrs['omega_l']
h = snap['universe'].attrs['hubble']

scale_factor = 1.0 / (1.0 + z)

In [6]:
print(shape)
print(size)

[1024 1024 1024]
[21.09375 21.09375 21.09375]


In [5]:
snap.close()